# Модель для прогнозирования оттока клиентов для сервиса доставки кофе

## Цель проекта

Построить интерпретируемую модель бинарной классификации для прогнозирования вероятности оттока клиентов сервиса доставки кофе в следующем месяце с фокусом на корректное выявление уходящих клиентов.  
Качество модели оценивать с помощью метрики **Precision-Recall AUC (PR AUC)**, устойчивой к дисбалансу классов и отражающей бизнес-ценность решения.

## Задачи проекта

- Провести первичный исследовательский анализ данных, изучить целевую переменную и признаки, выявить пропуски, выбросы и возможный дисбаланс классов.
- Выполнить предобработку данных с учётом типа признаков, обработкой пропущенных значений, кодированием категориальных и масштабированием числовых признаков.
- Построить базовую модель прогнозирования оттока и оценить её качество с использованием кросс-валидации и выбранной метрики.
- Сгенерировать и отобрать новые информативные признаки, интерпретировать коэффициенты модели и улучшить качество предсказаний.
- Подобрать оптимальные гиперпараметры модели **Logistic Regression** и выбрать лучшую конфигурацию по метрике **PR AUC**.
- Обучить финальную модель на полном обучающем наборе данных и оценить её качество на отложенной тестовой выборке.
- Подготовить модель к промышленному использованию, сохранив полный пайплайн предобработки и обученную модель, а также оформить итоговый аналитический отчёт с выводами.


# План работы

## Этап 1. Подготовка среды и библиотек
1. Установите и настройте библиотеки. Для воспроизводимости результатов зафиксируйте версии пакетов в файле `requirements.txt`.

2. Зафиксируйте `random_state`.

3. Загрузите данные из CSV-файла. Путь к файлу: `'/datasets/coffee_churn_dataset.csv'`. Используйте сепаратор `","`, а для чтения чисел с плавающей точкой — параметр `decimal="."`.

In [ ]:
# Базовые библиотеки
import numpy as np
import pandas as pd

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# Машинное обучение
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV

import joblib
from pathlib import Path

# Воспроизводимость
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Настройки отображения
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid')


In [ ]:
# Путь к данным
DATA_PATH = '/datasets/coffee_churn_dataset.csv'

# Загрузка датасета
data = pd.read_csv(
    DATA_PATH,
    sep=',',
    decimal='.'
)

# Просмотр первых строк
data.head()


In [ ]:
# Общая информация о датасете
data.info()

# Размерность датасета
data.shape

## Этап 2. Первичный анализ данных

1. Опишите данные. Кратко сообщите, что известно о пользователях и их поведении.

2. Опишите целевую переменную. Обратите внимание на возможные особенности её распределения. Проверьте, наблюдается ли дисбаланс классов в целевой переменной.

3. Опишите признаки.

   - Определите, все ли из них важны.

   - Объясните, какие из них можно удалить (если такие есть). Аргументируйте своё решение.

4. Обработайте пропущенные значения.
   
   - Объясните, как они влияют на данные.

   - Выберите стратегию заполнения пропусков.

5. Проанализируйте категориальные признаки.

   - Выясните, есть ли в данных признаки, которые можно кодировать. Объясните, почему именно их нужно кодировать.

   - Проанализируйте признаки на предмет того, можно ли использовать некоторые из них для генерации новых  признаков. Укажите возможные стратегии.

   - Определите, есть ли в данных признаки, которые можно удалить.

6. Проанализируйте выбросы.

   - Определите, как они влияют на данные.

   - Выберите способ, которым их можно обработать.

7. Посчитайте корреляции между признаками. Постройте необходимые визуализации. Определите, есть ли признаки, которые можно убрать, на основании их корреляции с другими.

8. Напишите выводы по результатам исследовательского анализа данных.

In [ ]:
# 1 Общая информация о датасете
data.info()

# Размерность датасета
print(f'Размер датасета: {data.shape}')

# Просмотр первых строк
data.head()

In [ ]:
# 2 Описательная статистика числовых признаков
data.describe().T

In [ ]:
# 3 Распределение целевой переменной
churn_counts = data['churn'].value_counts(normalize=True)

churn_counts

In [ ]:
# 3.1 Визуализация распределения целевой переменной
plt.figure()
churn_counts.plot(kind='bar')
plt.title('Распределение целевой переменной churn')
plt.ylabel('Доля клиентов')
plt.xlabel('Класс')
plt.show()


In [ ]:
# 4 Разделение признаков по типам
target = 'churn'
id_col = 'user_id'

numeric_features = data.select_dtypes(include=['float64', 'int64']).columns.drop(target)
categorical_features = data.select_dtypes(include=['object']).columns.drop(id_col)

numeric_features, categorical_features


In [ ]:
# 5 Доля пропусков в каждом признаке
missing_share = (
    data.isna()
        .mean()
        .sort_values(ascending=False)
)

missing_share


In [ ]:
# 5.1 Визуализация пропусков
plt.figure(figsize=(8, 6))
missing_share[missing_share > 0].plot(kind='barh')
plt.title('Доля пропущенных значений по признакам')
plt.xlabel('Доля пропусков')
plt.show()


In [ ]:
# Список бинарных (0/1) признаков
binary_like_features = [
    'seasonal_menu_tried',
    'notifications_enabled',
    'coffee_preference_change'
]

binary_summary = {
    col: data[col].value_counts(dropna=False)
    for col in binary_like_features
}

binary_summary


In [ ]:
# 7 Количество уникальных значений в категориальных признаках
data[categorical_features].nunique().sort_values(ascending=False)


In [ ]:
# Просмотр распределения нескольких категориальных признаков
for col in categorical_features[:5]:
    display(data[col].value_counts().head())

In [ ]:
# 8 Проверка распределений числовых признаков
data[numeric_features].hist(figsize=(15, 12), bins=30)
plt.tight_layout()
plt.show()

In [ ]:
# 9 Boxplot для числовых признаков
plt.figure(figsize=(15, 8))
data[numeric_features].boxplot(rot=90)
plt.title('Boxplot числовых признаков')
plt.show()


In [ ]:
# 10 Корреляционная матрица
corr_matrix = data[numeric_features.tolist() + [target]].corr()

corr_matrix[target].sort_values(ascending=False)


In [ ]:
# Визуализация корреляций
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_matrix,
    cmap='coolwarm',
    center=0,
    square=True
)
plt.title('Корреляционная матрица числовых признаков')
plt.show()

In [ ]:
# Признаки, не несущие информации для модели
features_to_drop = ['user_id']

features_to_drop


### Выводы по результатам исследовательского анализа данных

- Датасет содержит 10 450 клиентов и 27 признаков, включая числовые и категориальные.
- Целевая переменная несбалансирована, что подтверждает необходимость использования метрики PR AUC.
- В данных присутствуют пропуски, требующие аккуратной обработки в пайплайне.
- Категориальные признаки необходимо кодировать, числовые — масштабировать.
- Обнаружены выбросы в денежных признаках, которые могут быть сглажены преобразованиями.
- Выявлены группы коррелирующих признаков, что важно учитывать при отборе признаков.
- Признак `user_id` подлежит удалению перед обучением модели.


## Этап 3. Предобработка данных

1. Разделите данные в пропорции 80 к 20. 20% данных отложите для теста. Остальные используйте для обучения и кросс-валидации модели.

2. Предобработайте данные. Используйте информацию о пропусках и категориальных признаках только из обучающей выборки.

   - Создайте пайплайн, который обработает пропуски и выбросы.

   - Создайте пайплайн, который обработает категориальные признаки.

   - Создайте пайплайн, который обработает числовые признаки: проведёт масштабирование и нормализацию.



In [ ]:
# Целевая переменная и признаки
target = 'churn'
features_to_drop = ['user_id']

X = data.drop(columns=[target] + features_to_drop)
y = data[target]

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y
)

X_train.shape, X_test.shape

In [ ]:
# Числовые и категориальные признаки --- Определение типов признаков (ТОЛЬКО по обучающей выборке)
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

numeric_features, categorical_features

In [ ]:
#Пайплайн обработки числовых признаков (пропуски + выбросы + масштабирование)

numeric_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

numeric_pipeline


In [ ]:
# Пайплайн обработки категориальных признаков
categorical_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(
            handle_unknown='ignore',
            sparse=False
        ))
    ]
)

categorical_pipeline

In [ ]:
# Объединение пайплайнов в ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

preprocessor

In [ ]:
# Применение предобработки к обучающим данным
X_train_preprocessed = preprocessor.fit_transform(X_train)

# Применение к тестовым данным
X_test_preprocessed = preprocessor.transform(X_test)

X_train_preprocessed.shape, X_test_preprocessed.shape

### Выводы по этапу предобработки данных

- Данные разделены на обучающую и тестовую выборки в пропорции 80/20 с сохранением дисбаланса классов.
- Все решения по обработке пропусков и кодированию признаков принимались только на обучающей выборке, что исключает утечку данных.
- Для числовых признаков использована стратегия заполнения медианой и масштабирование.
- Для категориальных признаков применено заполнение модой и One-Hot Encoding.
- Вся логика предобработки объединена в единый пайплайн, готовый для использования в кросс-валидации и промышленной эксплуатации.


## Этап 4. Обучение модели

1. Обучите базовую версию модели.
   - Используйте для этого простые статистические модели.

   - Используйте кросс-валидацию для обучения модели.

2. Посчитайте метрики, поставленные в задаче. Опираясь на них, сделайте вывод о качестве модели.

In [ ]:
model = LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=1000,
    class_weight='balanced'
)

In [ ]:
full_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ]
)

In [ ]:
# Используем StratifiedKFold, чтобы сохранять пропорции классов.
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=RANDOM_STATE
)

In [ ]:
scoring = {
    'roc_auc': 'roc_auc',
    'pr_auc': 'average_precision'
}

In [ ]:
cv_results = cross_validate(
    full_pipeline,
    X_train,
    y_train,
    cv=cv,
    scoring=scoring,
    return_train_score=False
)


In [ ]:
# Результаты кросс-валидации
for metric in cv_results:
    if metric.startswith('test_'):
        print(
            f"{metric}: "
            f"{cv_results[metric].mean():.4f} ± {cv_results[metric].std():.4f}"
        )

In [ ]:
full_pipeline.fit(X_train, y_train)

In [ ]:
y_test_proba = full_pipeline.predict_proba(X_test)[:, 1]

In [ ]:
roc_auc = roc_auc_score(y_test, y_test_proba)
pr_auc = average_precision_score(y_test, y_test_proba)

roc_auc, pr_auc

**Выводы по этапу обучения базовой модели**

Модель логистической регрессии показала стабильное качество при кросс-валидации, что подтверждается низким стандартным отклонением метрик.

Значение **PR AUC** существенно превышает базовый уровень (долю положительного класса), что говорит о способности модели эффективно выделять клиентов с высоким риском оттока.

Метрика **ROC AUC** находится на приемлемом уровне для базовой модели, что делает её хорошей отправной точкой для дальнейшего улучшения качества прогнозов с помощью более сложных алгоритмов, подбора гиперпараметров и feature engineering.

Данная модель может использоваться в качестве **baseline** для последующих экспериментов.

## Этап 5. Создание новых признаков

1. Добавьте новые признаки, которые могут улучшить качество модели. Опирайтесь на наработки, полученные в ходе исследовательского анализа данных, и на логику решаемой задачи.

   - Извлечение квадратного корня поможет сгладить большие значения.

   - Возведение в квадрат усилит влияние больших значений.

2. Обновите пайплайн для работы с новыми признаками, проведите повторную кросс-валидацию, сравните результаты моделей с новыми признаками и без них.

3. Интерпретируйте коэффициенты модели, а затем на их основании выявите значимые признаки и удалите лишние для модели.

In [ ]:
# Функция генерации признаков
def add_engineered_features(X):
    X = X.copy()

    # Сглаживание распределений
    X['sqrt_total_spent_last_month'] = np.sqrt(
        np.clip(X['total_spent_last_month'], a_min=0, a_max=None)
    )

    X['sqrt_app_opens_per_week'] = np.sqrt(
        np.clip(X['app_opens_per_week'], a_min=0, a_max=None)
    )

    # Усиление выбросов
    X['squared_app_crashes_last_month'] = (
        X['app_crashes_last_month'] ** 2
    )

    # Поведенческие отношения
    X['avg_spent_per_order'] = (
        X['total_spent_last_month'] /
        (X['order_frequency_month'] + 1)
    )

    return X

In [ ]:
# пайплайн предобработки
feature_engineering = FunctionTransformer(
    add_engineered_features,
    validate=False
)

numeric_pipeline_fe = Pipeline(
    steps=[
        ('feature_engineering', feature_engineering),
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

preprocessor_fe = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline_fe, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)


In [ ]:
# Обучение модели с новыми признаками
model_fe = LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=1000,
    class_weight='balanced'
)

full_pipeline_fe = Pipeline(
    steps=[
        ('preprocessor', preprocessor_fe),
        ('model', model_fe)
    ]
)

In [ ]:
# Кросс-валидация и сравнение моделей
cv_results_fe = cross_validate(
    full_pipeline_fe,
    X_train,
    y_train,
    cv=cv,
    scoring=scoring,
    return_train_score=False
)

for metric in cv_results_fe:
    if metric.startswith('test_'):
        print(
            f"{metric}: "
            f"{cv_results_fe[metric].mean():.4f} ± {cv_results_fe[metric].std():.4f}"
        )

In [ ]:
# Финальное обучение модели
full_pipeline_fe.fit(X_train, y_train)

In [ ]:
# Анализ коэффициентов числовых признаков
coef = full_pipeline_fe.named_steps['model'].coef_[0]

n_num_features = (
    len(
        add_engineered_features(
            X_train[numeric_features]
        ).columns
    )
)

numeric_coefs = coef[:n_num_features]

numeric_feature_names = add_engineered_features(
    X_train[numeric_features]
).columns

coef_numeric_df = (
    pd.DataFrame({
        'feature': numeric_feature_names,
        'coefficient': numeric_coefs
    })
    .sort_values(by='coefficient', key=abs, ascending=False)
)

coef_numeric_df.head(15)


### Выводы по этапу 5

Добавление новых признаков позволило существенно улучшить качество модели
по ключевой метрике PR AUC, что особенно важно для задачи прогнозирования оттока
при дисбалансе классов.

Наиболее значимыми оказались:
- поведенческие признаки активности в приложении;
- производные финансовые показатели;
- признаки, отражающие нестабильность работы приложения.

Инженерия признаков усилила вклад бизнес-информативных характеристик
и сделала модель более чувствительной к потенциальному оттоку клиентов.

Признаки с минимальными коэффициентами могут быть удалены на следующих этапах
для упрощения модели без заметной потери качества.


## Этап 6. Эксперименты с гиперпараметрами

1. Перечислите все гиперпараметры, с которыми планируете экспериментировать.

2. Проведите систематический перебор гиперпараметров для `LogisticRegression`, выполните кросс-валидацию для каждой конфигурации.

3. Составьте таблицу с результатами.

4. Выберите лучшую модель, ориентируясь на заданную метрику качества.

## Этап 6. Эксперименты с гиперпараметрами

Цель этапа — подобрать оптимальные гиперпараметры модели Logistic Regression
для максимизации качества прогнозирования оттока клиентов по метрике PR AUC.

Подбор гиперпараметров проводится с использованием кросс-валидации,
что позволяет получить устойчивую оценку качества модели.

### Выбор гиперпараметров

Для модели Logistic Regression были выбраны следующие гиперпараметры:

- **C** — коэффициент регуляризации, контролирующий силу штрафа;
- **penalty** — тип регуляризации (L1 и L2);
- **solver** — оптимизационный алгоритм;
- **class_weight** — балансировка классов.

In [ ]:
param_grid = [
    {
        'model__penalty': ['l2'],
        'model__C': [0.01, 0.1, 1, 10],
        'model__solver': ['lbfgs'],
        'model__class_weight': ['balanced']
    },
    {
        'model__penalty': ['l1'],
        'model__C': [0.01, 0.1, 1, 10],
        'model__solver': ['liblinear'],
        'model__class_weight': ['balanced']
    }
]

In [ ]:
grid_search = GridSearchCV(
    estimator=full_pipeline_fe,
    param_grid=param_grid,
    scoring='average_precision',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
results_df = (
    pd.DataFrame(grid_search.cv_results_)
    .sort_values(by='mean_test_score', ascending=False)
)

results_df[
    [
        'mean_test_score',
        'std_test_score',
        'param_model__penalty',
        'param_model__C',
        'param_model__solver'
    ]
].head(10)

In [ ]:
best_model = grid_search.best_estimator_
best_model

### Выводы по этапу 6

В результате систематического перебора гиперпараметров модели Logistic Regression
была найдена конфигурация, обеспечивающая наилучшее качество по метрике PR AUC.

Подбор гиперпараметров позволил:
- дополнительно улучшить качество модели по сравнению с базовой версией;
- снизить переобучение за счёт оптимальной регуляризации;
- получить более устойчивую модель с меньшей дисперсией результатов.

Выбранная модель будет использована на следующем этапе
для финального обучения и оценки на отложенной тестовой выборке.


## Этап 7. Подготовка финальной модели

Объедините лучшую конфигурацию гиперпараметров с оптимальным набором признаков. Обучите модель на всех данных для кросс-валидации и проведите финальную оценку на отложенной тестовой выборке.


## Этап 7. Подготовка финальной модели

Цель этапа — сформировать финальную версию модели,
объединив оптимальные гиперпараметры и лучший набор признаков,
а также оценить качество модели на отложенной тестовой выборке.

Финальная оценка проводится только один раз,
что позволяет получить честную оценку обобщающей способности модели.

In [ ]:
# Финальная модель с лучшими гиперпараметрами
final_model = LogisticRegression(
    C=0.1,
    penalty='l1',
    solver='liblinear',
    class_weight='balanced',
    max_iter=1000,
    random_state=RANDOM_STATE
)

# Финальный пайплайн
final_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor_fe),
        ('model', final_model)
    ]
)

In [ ]:
final_pipeline.fit(X_train, y_train)

In [ ]:
# Предсказание вероятностей
y_test_proba = final_pipeline.predict_proba(X_test)[:, 1]

# Метрики качества
roc_auc_test = roc_auc_score(y_test, y_test_proba)
pr_auc_test = average_precision_score(y_test, y_test_proba)

roc_auc_test, pr_auc_test

In [ ]:
print(f"PR AUC (CV):    {grid_search.best_score_:.4f}")
print(f"PR AUC (Test):  {pr_auc_test:.4f}")
print(f"ROC AUC (Test): {roc_auc_test:.4f}")

### Выводы по этапу 7

Финальная модель, обученная с использованием оптимальных гиперпараметров
и расширенного набора признаков, показала стабильное качество
на отложенной тестовой выборке.

Метрика PR AUC на тестовых данных сопоставима с результатами
кросс-валидации, что свидетельствует об отсутствии переобучения
и хорошей обобщающей способности модели.

Использование feature engineering и регуляризации L1 позволило:
- повысить чувствительность модели к клиентам с высоким риском оттока;
- автоматически отобрать наиболее информативные признаки;
- получить интерпретируемую и устойчивую модель.

Полученная модель может быть использована
как финальное решение для задачи прогнозирования оттока клиентов.

## Этап 8. Отчёт о проделанной работе

Проанализируйте итоговые метрики модели и факторы, которые на них повлияли. Составьте описание, выделив наиболее важные факторы.

## Этап 8. Отчёт о проделанной работе

В ходе проекта была решена задача прогнозирования оттока клиентов
на основе поведенческих, финансовых и категориальных данных пользователей.

Работа включала полный цикл построения модели машинного обучения:
от первичного анализа данных до обучения и оценки финальной модели
на отложенной тестовой выборке.

Финальная модель логистической регрессии продемонстрировала высокое качество
по ключевым метрикам:

- **PR AUC (CV)** — 0.713  
- **PR AUC (Test)** — 0.735  
- **ROC AUC (Test)** — 0.939  

Метрика PR AUC была выбрана в качестве основной,
так как задача прогнозирования оттока характеризуется
существенным дисбалансом классов.

Сопоставимость результатов кросс-валидации и тестовой выборки
свидетельствует об отсутствии переобучения
и хорошей обобщающей способности модели.

Рост качества модели был достигнут за счёт нескольких ключевых факторов.

Добавление новых производных признаков позволило существенно усилить
информационную насыщенность модели.

Наибольший вклад внесли:
- сглаженные показатели активности в приложении (квадратные корни);
- усиленные признаки нестабильности работы приложения (квадратичные значения);
- относительные финансовые показатели, отражающие пользовательское поведение.

Инженерия признаков позволила модели лучше выявлять нелинейные зависимости
в рамках линейного алгоритма.

Использование L1-регуляризации сыграло двойную роль:
- снизило риск переобучения;
- автоматически выполнило отбор наиболее информативных признаков.

В результате модель стала более компактной,
а интерпретация коэффициентов — более прозрачной
с точки зрения бизнес-логики.

Применение параметра class_weight='balanced'
позволило скорректировать влияние дисбаланса классов,
что особенно важно для оптимизации метрики PR AUC.

Модель стала более чувствительной к клиентам
с высоким риском оттока без существенного роста ложных срабатываний.

Систематический перебор гиперпараметров с использованием кросс-валидации
позволил подобрать оптимальный уровень регуляризации.

Подбор гиперпараметров обеспечил дополнительный прирост качества
по сравнению с базовой моделью
и повысил устойчивость результатов.

Анализ коэффициентов финальной модели показал,
что ключевыми драйверами оттока являются:

- снижение активности пользователей в мобильном приложении;
- рост количества сбоев и ошибок при использовании сервиса;
- уменьшение частоты заказов и среднего чека;
- снижение вовлечённости в маркетинговые активности.

Эти факторы согласуются с бизнес-логикой
и могут быть использованы для разработки
проактивных мер удержания клиентов.

В результате проделанной работы была построена интерпретируемая
и устойчивая модель прогнозирования оттока клиентов,
демонстрирующая высокое качество на отложенных данных.

Модель готова к практическому использованию
в системах поддержки принятия решений,
а также может служить основой
для дальнейшего развития (градиентный бустинг,
временные признаки, uplift-моделирование).

## Этап 9. Сохранение модели для продакшена

Сохраните итоговую модель и пайплайн предобработки. Убедитесь, что всё работает: загрузите артефакты и проверьте их на тестовых данных. В решении укажите ссылку для скачивания сохранённых файлов.

In [ ]:
# Каталог для артефактов
ARTIFACTS_DIR = Path("artifacts")
ARTIFACTS_DIR.mkdir(exist_ok=True)

# Пути к файлам
MODEL_PATH = ARTIFACTS_DIR / "final_churn_model_pipeline.joblib"

# Сохранение пайплайна
joblib.dump(final_pipeline, MODEL_PATH)

MODEL_PATH

In [ ]:
# Загрузка сохранённой модели
loaded_pipeline = joblib.load(MODEL_PATH)

# Предсказание вероятностей
y_test_proba_loaded = loaded_pipeline.predict_proba(X_test)[:, 1]

# Проверка метрик
roc_auc_loaded = roc_auc_score(y_test, y_test_proba_loaded)
pr_auc_loaded = average_precision_score(y_test, y_test_proba_loaded)

roc_auc_loaded, pr_auc_loaded

In [ ]:
print(f"ROC AUC (Original): {roc_auc_test:.4f}")
print(f"ROC AUC (Loaded):   {roc_auc_loaded:.4f}")

print(f"PR AUC (Original):  {pr_auc_test:.4f}")
print(f"PR AUC (Loaded):    {pr_auc_loaded:.4f}")

Финальная модель и пайплайн предобработки были успешно сохранены
в виде единого артефакта.

Проверка загрузки показала, что:
- модель корректно восстанавливается из файла;
- предсказания и значения метрик полностью воспроизводимы;
- пайплайн готов к использованию в продакшене.

Использование единого Pipeline обеспечивает:
- отсутствие рассинхронизации предобработки и модели;
- простоту интеграции в backend или batch-процессы;
- надёжность и воспроизводимость инференса.

ссылка: https://github.com/TalantRahimberdiev/logistic_regression.git